In [1]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense, StringLookup
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.src.layers import LSTM

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
# Load data
def map_event(event):
    if event == 'view':
        return 1
    elif event == 'addtocart':
        return 2
    elif event == 'transaction':
        return 3
    else:
        return 0

basepath = '../../datasets/preprocessed_datasets/retailrocket/'
events = pd.read_pickle(basepath +'events.pkl')
events['event_mapped'] = events['event'].apply(map_event)
events = events.drop(columns=['timestamp', 'event', 'transactionid', 'datetime'])
events = events.sample(10000)
item_properties = pd.read_pickle(basepath +'item_data_extracted.pkl')
display(events, item_properties)

,visitorid,itemid,event_mapped
1153096,693935,213903,1
2662469,576413,179601,1
1640381,582544,154594,1
2077204,952532,313448,1
1106554,362051,96924,1
...,...,...,...
1446444,638482,460639,1
2010112,561735,46831,1
2613003,1158126,101967,1
424502,1132799,254432,1


,itemid,categoryid,available,properties,property_values
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189,...","[1276750, n3.168 1144008, n15360.000 628176 n1..."
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 2...","[769062, 1154859, 679677, 519769, 769062, 1726..."
2,2,1305,0,"[1063, 112, 159, 202, 282, 283, 318, 332, 364,...","[n396.000 145688, 679677, 519769, 648485 n6000..."
3,3,1171,0,"[1025, 1080, 112, 159, 202, 227, 250, 283, 30,...","[769062, 769062, 679677, 519769, 261419, 13057..."
4,4,1038,0,"[112, 115, 159, 202, 227, 28, 283, 33, 364, 59...","[679677, n24.000, 519769, 371058 71429, 588652..."
...,...,...,...,...,...
417048,466862,1376,0,"[112, 139, 159, 186, 202, 227, 28, 283, 29, 34...","[679677, 769062, 519769, 575816 n432.000, 7173..."
417049,466863,173,0,"[1014, 112, 159, 202, 227, 283, 364, 400, 47, ...","[1075463, 679677, 519769, 1194687 550565 24255..."
417050,466864,373,1,"[1036, 112, 152, 159, 202, 227, 230, 283, 348,...","[1154859, 679677, 1071492, 519769, 1262739 205..."
417051,466865,421,0,"[1031, 112, 159, 202, 227, 277, 28, 283, 332, ...","[1088309, 679677, 519769, 150169 780351 820477..."


In [4]:
# Initialize tokenizer
string_lookup = StringLookup()
string_lookup.adapt(pandas_interaction_data['productName'])

tokenizer = Tokenizer(num_words=string_lookup.vocabulary_size(), oov_token="<OOV>")
tokenizer.fit_on_texts(pandas_interaction_data['productName'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(pandas_interaction_data['productName'])

# Pad sequences
padded_sequences = pad_sequences(sequences, padding='post')

NameError: name 'pandas_interaction_data' is not defined

In [27]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values, 
    'product_color': pandas_interaction_data['color_mapped'].values, 
    'product_description': padded_sequences, 
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [28]:
interaction_dataset

<_TensorSliceDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(7,), dtype=tf.int32, name=None)}>

In [29]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(7,), dtype=tf.int32, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(7,), dtype=tf.int32, name=None)}>

In [30]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

unique_colors = np.array(product_data["color_mapped"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)

product_colors_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_colors_vocabulary.adapt(unique_colors)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [31]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, unique_product_color_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])

        # Color embeddings
        self.color_embedding = tf.keras.Sequential([
            unique_product_color_vocab,
            Embedding(unique_product_color_vocab.vocabulary_size(), embedding_dimension)
        ]) 


        # Textual description model
        self.textual_description_model = tf.keras.Sequential([
            Embedding(input_dim=unique_product_ids_vocab.vocabulary_size(), output_dim=embedding_dimension),
            LSTM(64),
            Dense(64, activation='relu')
        ])

        # New embeddings for color and description
        self.reduced_product_embeddings = Dense(64, activation='relu')
        self.reduced_description_embeddings = Dense(32, activation='relu')
        self.reduced_color_embeddings = Dense(32, activation='relu')
        
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, input_shape=(256,), activation="relu")

        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])

        color_embeddings = self.color_embedding(features["product_color"])
        description_embeddings = self.textual_description_model(features["product_description"])

        reduced_product_embeddings = self.reduced_product_embeddings(product_embeddings)
        reduced_description_embeddings = self.reduced_description_embeddings(description_embeddings)
        reduced_color_embeddings = self.reduced_color_embeddings(color_embeddings)


        # Now concatenate
        user_combined = tf.concat([user_embeddings, ], axis=1)  # shape will be [None, 128]
        product_combined = tf.concat([reduced_product_embeddings, reduced_color_embeddings, reduced_description_embeddings], axis=1)  # shape will be [None, 128]
        


        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_combined)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [32]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, product_colors_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [33]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3) 
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
7/7 [==============================] - 4s 248ms/step - auc: 0.0000e+00 - rmse: 0.9984 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0388 - factorized_top_k/top_5_categorical_accuracy: 0.0812 - factorized_top_k/top_10_categorical_accuracy: 0.1100 - factorized_top_k/top_50_categorical_accuracy: 0.2450 - factorized_top_k/top_100_categorical_accuracy: 0.3288 - loss: 493.4432 - regularization_loss: 0.0000e+00 - total_loss: 493.4432
Epoch 2/3
7/7 [==============================] - 2s 224ms/step - auc: 0.0000e+00 - rmse: 0.9758 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.3887 - factorized_top_k/top_5_categorical_accuracy: 0.5250 - factorized_top_k/top_10_categorical_accuracy: 0.5925 - factorized_top_k/top_50_categorical_accuracy: 0.7675 - factorized_top_k/top_100_categorical_accuracy: 0.8413 - loss: 491.5392 - regularization_loss: 0.0000e+00 - total_loss: 491.5392
Epoch 3/3
7/7 [=======

In [34]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

7/7 [==============================] - 2s 234ms/step - auc: 0.0000e+00 - rmse: 0.8845 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9887 - factorized_top_k/top_5_categorical_accuracy: 0.9950 - factorized_top_k/top_10_categorical_accuracy: 0.9962 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 486.9747 - regularization_loss: 0.0000e+00 - total_loss: 486.9747


([0.0,
  0.8844582438468933,
  0.0,
  0.0,
  0.9887499809265137,
  0.9950000047683716,
  0.9962499737739563,
  1.0,
  1.0,
  109.08023071289062,
  0,
  109.08023071289062],
 [0.0, 0.0, 0.0])

In [35]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

4/4 [==============================] - 1s 261ms/step - auc: 0.0000e+00 - rmse: 0.9491 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.7450 - factorized_top_k/top_5_categorical_accuracy: 0.8000 - factorized_top_k/top_10_categorical_accuracy: 0.8450 - factorized_top_k/top_50_categorical_accuracy: 0.8900 - factorized_top_k/top_100_categorical_accuracy: 0.9100 - loss: 166.3419 - regularization_loss: 0.0000e+00 - total_loss: 166.3419


([0.0,
  0.9491428136825562,
  0.0,
  0.0,
  0.7450000047683716,
  0.800000011920929,
  0.8450000286102295,
  0.8899999856948853,
  0.9100000262260437,
  16.64014434814453,
  0,
  16.64014434814453],
 [0.0, 0.0, 0.0])

In [36]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		0.9491428136825562
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		0.7450000047683716
factorized_top_k/top_5_categorical_accuracy		0.800000011920929
factorized_top_k/top_10_categorical_accuracy		0.8450000286102295
factorized_top_k/top_50_categorical_accuracy		0.8899999856948853
factorized_top_k/top_100_categorical_accuracy		0.9100000262260437
AUC: 0.0


In [37]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([10317123,  9045476,  3583920, 10764275,  6985511,  5169066,
        6400396,  9734712,  6101544,  7083821,  2216762,  4520433,
        2221250, 11002672,  1960889, 10690262,  8830713, 10109551,
        9368658,  1963512,  4660874, 10014913, 10573645,  5587005,
        8704142, 11028047,  2177070,  2071053,  3992822,  6994391,
        6912252,  5449778,  6416070,  1993271,  7916969,  8256388,
        6072041, 10858367,  6520387,  4054179, 11061968,  6591921,
        5649813, 10843008,  1960627, 10622816,  5013630,  7600040,
        4601491,  8339910,  7741676,  4723988,  7615510,  6751909,
        7146808,  3544851,  6403624,  7587086,  6569161,  2208487,
        5976793,  5358389,  8375789,  2236422])

In [38]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  10317123
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
119,3591582,4062862317623,Sportliche Ballerina Rauleder beige,beige,52.621.40,6590678,315571,w,SK,H-W,...,39.0,6.0,10.0,0.0,Gummi,0.0,0.0,"[3591582, 3591640, 3591580, 3591642, 3591581, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",6
261,3845686,4062862157472,Winterstiefel Glattleder schwarz,schwarz,51.619.87,6590677,315591,w,PT,H-W,...,37.5,4.5,7.0,42.0,TR,0.0,1.0,"[3845686, 3849370, 3849367, 3845695, 3847291, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
301,3846740,4060666203982,Spangenpump Textil rosa,rosa,01.363.64,6590677,315561,w,PT,F-S|H-W,...,39.0,6.0,10.0,0.0,TPU,0.0,1.0,"[3846740, 3846742, 3846735, 3846736, 3846741, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",8
387,3696734,4062862315391,PG0460 Sneaker high Rauleder grau,grau,0460.50.03,3692257,3692263,m,PT,H-W,...,45.0,10.5,12.0,9.0,TR,0.0,0.0,"[3696734, 3695640, 3696732, 3696729, 3695634, ...","[39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44....",4
456,4075066,4062862370765,Schnürstiefelette Lackleder blau,blau,52.746.26,6590678,315587,w,SK,H-W,...,35.0,2.5,3.0,20.0,Gummi,0.0,0.0,"[4075066, 4075072, 4075077, 4075079, 4075076, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
480,4093082,4062862436720,Elegante Stiefelette Lackleder schwarz,schwarz,52.791.87,6590678,5014852,w,SK,H-W,...,43.0,9.0,16.0,17.0,TR,0.0,0.0,"[4093082, 4093072, 4093077, 4093071, 4093080, ...","[35.0, 35.5, 36.0, 37.0, 38.0, 38.5, 39.0, 40....",0
582,3693588,4062862032540,Sneaker low Rauleder,,56.964.33,4997827,315567,w,VN,H-W,...,42.0,8.0,14.0,0.0,Gummi-EVA,0.0,0.0,"[3693588, 4846740, 3763972, 4823173, 3591778, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",15
1218,6113376,4064032454244,Biker- / Combat Boot Rauleder braun,braun,72.704.41,6590678,5014852,w,PT,H-W,...,39.0,6.0,10.0,18.0,TR,0.0,0.0,"[6113376, 6116055, 6116347, 6113377, 6113382, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
1326,6114331,4064032421734,Sportlicher Stiefel Glattleder braun,braun,72.799.24,6590678,5014850,w,SK,H-W,...,36.0,3.5,5.0,42.0,TR,1.0,1.0,"[6114331, 6112111, 6639919, 6112110, 6639915, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
1732,7593990,4064032142172,Keilsandalette Glattleder multicolour,multicolour,82.759.20,6590678,315564,w,PT,F-S,...,43.0,9.0,16.0,0.0,TR,0.0,0.0,"[7593990, 7593991, 7593978, 7831468, 7831469, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",10


scores:  [0.02560227 0.02115727 0.02111601 0.01942505 0.01909895 0.01834647
 0.01769243 0.01744029 0.01730673 0.01647986]


In [39]:
user_id = next(iter(train.take(1)))['customer_id'].numpy()
predict_user(user_id)

predicting user:  6909647
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
101,3504775,4062862281702,Chelsea Boot Rauleder braun,braun,56.661.32,6590678,315582,w,PT,H-W,...,40.5,7.0,12.0,11.0,TR,0.0,0.0,"[3504775, 3477007, 3504771, 3478521, 3478174, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
19,549066,4054452850924,Keilpumps Rauleder schwarz,schwarz,02.690.47,6590678,315588,w,SK,F-S|H-W,...,39.0,6.0,10.0,0.0,TPU,0.0,0.0,"[549066, 549072, 549070, 549063, 549069, 54907...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
21,407456,4054452837239,Pantoletten Rauleder grau,grau,03.705.13,6590677,315572,w,SK,F-S,...,40.0,6.5,8.0,0.0,Gummi,0.0,0.0,"[407456, 407457, 407454, 407458, 407453, 40745...","[36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43....",4
61,555008,4054452768687,Eleganter Pump Glattleder blau,blau,05.180.36,6590677,315573,w,SK,F-S|H-W,...,43.0,9.0,16.0,0.0,EVA,0.0,0.0,"[555008, 3511789, 3511790, 3511787, 3511785, 3...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",1
263,3842359,4062862266945,Winterstiefel Glattleder braun,braun,51.615.84,6590677,315591,w,PT,H-W,...,39.0,6.0,10.0,41.0,TR,0.0,1.0,"[3842359, 3866739, 3866742, 3866731, 3844022, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
602,4770728,4062862889762,Riemchensandale Rauleder blau,blau,61.723.16,6590677,315563,w,PT,F-S,...,40.5,7.0,12.0,0.0,TPU,0.0,1.0,"[4770728, 4777412, 4772440, 4772439, 4772442, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
730,4847355,4062862647416,Keilsandalette Glattleder braun,braun,62.043.13,6590678,315564,w,PT,F-S,...,39.0,6.0,10.0,0.0,TR,0.0,0.0,"[4847355, 4777312, 4770237, 4771075, 4847356, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
782,4822647,4062862606628,Mokassin Rauleder grün,grün,63.337.19,6590677,315590,w,SK,F-S,...,35.5,3.0,4.0,0.0,TR,0.0,0.0,"[4822647, 4822651, 4822650, 4822649, 4822648, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",11
1258,6113501,4064032417799,Sportliche Stiefelette Glattleder blau,blau,72.718.66,6590678,5014851,w,PT,H-W,...,43.0,9.0,16.0,13.0,Gummi,1.0,1.0,"[6113501, 6113495, 6113496, 6113490, 6113497, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",1
1312,6142500,4064032542590,Eleganter Stiefel Glattleder,,75.628.21,6590677,315585,w,PT,H-W,...,38.0,5.0,8.0,40.0,TPU,0.0,0.0,"[6142500, 6892638, 6892634, 6902873, 6846954, ...","[35.5, 37.0, 37.5, 38.0, 39.0, 40.0, 40.5, 41....",15
1546,6177984,4064032598566,PG8000 Sneaker low Glattleder schwarz,schwarz,8000.12.05,4997827,3692261,m,CN,H-W,...,43.0,9.0,9.0,0.0,Gummi-EVA,0.0,0.0,"[6177984, 6177982, 6177981, 6177986, 6177985, ...","[40.5, 41.0, 42.0, 43.0, 44.0, 44.5, 45.0, 46....",0


scores:  [0.03588894 0.0326204  0.03164598 0.03030481 0.02852748 0.02564616
 0.02534861 0.02402134 0.02290115 0.02228493]


In [40]:
model.save('../../models/gabor/gabor_tensorflow_reco_all_data_3_epochs_v1.h5')

/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.